In [7]:
import pandas as pd
import numpy as np
from random import random

from CTL2.causal_tree_learn import CausalTree
from DGP.DGP import dgp2
from DGP.DGP import dgp4
from DGP.DGP import dgp4int
from DGP.DGP import dgp8

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

import matplotlib.pyplot as plt
import pickle as pkl
#from notify_run import Notify 
#notify = Notify()
import datetime

from utility.utility import transposer as tp
from utility.utility import writer
from utility.utility import reader

### Parameters:

### ATE2

In [3]:
def mc_ate8(n, tsize, var_e, reps, nomin_test):
    #true ATE
    ATE1 = np.ones(reps)*5
    ATE2 = np.ones(reps)*-1
    ATE3 = np.ones(reps)*2.5
    ATE4 = np.ones(reps)*-3.5
    ATE5 = np.ones(reps)*3.5
    ATE6 = np.ones(reps)*-2.5
    ATE7 = np.ones(reps)*1
    ATE8 = np.ones(reps)*-5
    #predicted ate
    ate1 = []
    ate2 = []
    ate3 = []
    ate4 = []
    ate5 = []
    ate6 = []
    ate7 = []
    ate8 = []
    #ITE dummy
    tauhat = np.zeros((tsize, 1))
    #true ITE

    for i in range(reps):
        x_train, x_test, y_train, y_test, treat_train, treat_test = dgp8(n, tsize, var_e)
        tau = np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0)),5, 
                           np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0)), -1, 
                                    np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0)), 2.5, 
                                             np.where(((x_test[:,0] < 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0)), -3.5, 
                                                      np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0)), 3.5, 
                                                               np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0)), -2.5, 
                                                                        np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] < 0)), 1, -5)))))))
        ctl = CausalTree(honest=True, weight=0.0, split_size=0.0, max_depth = 3) #which type of tree to call
        ctl.fit(x_train, y_train, treat_train, nomin_test = nomin_test) #select est size when fitting
        ctl.prune()
        ctl_predict = ctl.predict(x_test)
        #predicted ATE
        ate1 = np.append(ate1, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0))]))
        ate2 = np.append(ate2, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0))]))
        ate3 = np.append(ate3, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0))]))
        ate4 = np.append(ate4, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0))]))
        ate5 = np.append(ate5, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0))]))
        ate6 = np.append(ate6, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0))]))
        ate7 = np.append(ate7, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] < 0))]))
        ate8 = np.append(ate8, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] < 0) & (x_test[:,2] < 0))]))
        #predicted ITE
        tauhat = np.append(tauhat, ctl_predict.reshape(-1,1), axis = 1)

    #ATE MSE's
    mean_ate = np.array([np.mean(ate1), np.mean(ate2), np.mean(ate3), np.mean(ate4), np.mean(ate5), np.mean(ate6), np.mean(ate7), np.mean(ate8)])    
    mse_ate = np.array([mse(ATE1, ate1), mse(ATE2, ate2),mse(ATE3, ate3), mse(ATE4, ate4),mse(ATE5, ate5), mse(ATE6, ate6), mse(ATE7, ate7), mse(ATE8, ate8)])
    bias_ate = np.array([np.sum(np.subtract(ATE1,ate1))/reps, np.sum(np.subtract(ATE2,ate2))/reps, np.sum(np.subtract(ATE3,ate3))/reps, np.sum(np.subtract(ATE4,ate4))/reps,
                         np.sum(np.subtract(ATE5,ate5))/reps, np.sum(np.subtract(ATE6,ate6))/reps, np.sum(np.subtract(ATE7,ate7))/reps, np.sum(np.subtract(ATE8,ate8))/reps])
    var_ate = np.array([np.var(ate1), np.var(ate2), np.var(ate3), np.var(ate4), np.var(ate5), np.var(ate6), np.var(ate7), np.var(ate8)])

    #TOTAL MSE's
    ind_var = np.var(tauhat[:, 1:], axis = 1)
    ind_mean = np.mean(tauhat, axis = 1)

    total_var = np.sum(ind_var)/tsize
    total_bias = np.sum(np.square(np.subtract(ind_mean, tau)))/tsize
    total_mse = total_var + total_bias

    results = np.concatenate([np.array([nomin_test]), mean_ate, bias_ate, var_ate, mse_ate, np.array([total_bias]), np.array([total_var]), np.array([total_mse])])
    return results

In [5]:
n = 1000
tsize = 500
reps = 500
test_sizes = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
res = []
var = [0.01, 1.0, 2.5]

print("start time: ", str(datetime.datetime.now())[11:16])

for v in var:
    df = pd.DataFrame()
    for size in test_sizes:
        results = mc_ate8(n, tsize, v, reps, size)
        df = pd.concat([df, pd.DataFrame(results).transpose()])
    df.columns=['test_sizes', 'Mean1', 'Mean2', 'Mean3', 'Mean4','Mean5', 'Mean6', 'Mean7', 'Mean8', 
                'MSE1', 'MSE2','MSE3', 'MSE4','MSE5', 'MSE6','MSE7', 'MSE8',
                'BIAS1', 'BIAS2','BIAS3', 'BIAS4', 'BIAS5', 'BIAS6','BIAS7', 'BIAS8',
                'VAR1', 'VAR2','VAR3', 'VAR4',  'VAR5', 'VAR6','VAR7', 'VAR8', 
                'MSE_T_BIAS', 'MSE_T_VAR', 'MSE_TOTAL']    
    res.append([v])
    res.append([df])
    
print("end time: ", str(datetime.datetime.now())[11:16])

start time:  14:24
end time:  14:25


In [6]:
res

[[0.01],
 [   test_sizes     Mean1     Mean2     Mean3     Mean4     Mean5     Mean6  \
  0         0.2  3.802484 -2.625702  1.692068 -2.604767  3.546771 -2.597074   
  0         0.3  3.899774 -3.147856  1.938110 -3.214923  4.035132 -3.221379   
  0         0.4  3.780899 -2.928744  2.395660 -2.946008  3.106118 -2.946008   
  0         0.5  3.234358 -2.794856  2.253426 -2.812655  3.028914 -2.844154   
  0         0.6  3.805707 -2.951510  2.827460 -2.951510  3.342049 -2.951510   
  0         0.7  3.243876 -2.697568  2.316220 -2.760747  2.979631 -2.716673   
  0         0.8  3.788908 -2.638678  1.818962 -2.638678  3.691480 -2.569872   
  
        Mean7     Mean8      MSE1  ...      VAR2      VAR3      VAR4      VAR5  \
  0  1.779823 -2.637897  1.197516  ...  2.825797  1.377523  1.027093  0.137303   
  0  1.991465 -3.233994  1.100226  ...  4.705189  0.420225  0.156544  0.621482   
  0  2.192033 -2.946008  1.219101  ...  3.762799  0.363320  0.351362  0.492190   
  0  2.040413 -2.856771  1.7